In [1]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from openai import OpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-mini")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
def load(url): 
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("post-content", "post-title", "post-header")
            )
        ),
    )
    docs = loader.load()

    return docs[0]

In [3]:
def create_retriever():
    urls = [
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
        "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    ]
    
    docs = []
    
    for url in urls:
        docs.append(load(url))
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))
    # retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
    retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 6})
    return retriever

In [4]:
def search_retriever(question, retriever):
    retriver_result = retriever.invoke(question)
    return str(retriver_result), ""

In [5]:
def relevance_checker(question, docs):

    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "docs의 page_content 내용 중에 qeustion과 관련이 있는 정보만 뽑아줘. 리스트 형태로 들어오고, 관련이 있으면 o, 없으면 x로 출력해줘. 판단할 때 굉장히 넓고 자비로운 마음으로 너무 빡빡하지 않게 평가해줘 ex) [o, o, o, o, x, o]"
            }, 
            {
                "role": "user",
                "content": 'question: ' + question + '\ndocs:' + docs
            }
        ]
    )

    flag = response.choices[0].message.content
    print(flag)
    

    parser = JsonOutputParser()
    prompt = PromptTemplate(
        template="""
하나라도 x가 존재한다면 no, 모두 o라면 yes라고 답변해줘. 
만약 데이터가 없다면 no로 답변해줘.

Answer the user query.\n{format_instructions}\n{query}\n
key는 related로 해줘
    """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    chain = prompt | llm | parser
    
    result = chain.invoke({"query": flag})
    return result

In [6]:
def select_relevance(question, docs):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "docs의 page_content 내용 중에 qeustion과 관련이 있는 정보만 뽑아줘. 리스트 형태로 들어오고, 관련이 있으면 o, 없으면 x로 출력해줘. ex) [o, o, o, o, x, o]"
            }, 
            {
                "role": "user",
                "content": 'question: ' + question + '\ndocs:' + docs
            }
        ]
    )

    flag = response.choices[0].message.content
    print(flag)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "flag가 o인 docs만 골라 배열 형태로 답변해줘.\n 출력 형식: [{source: ~~, page_content: ~}]\nflag:\n" + flag + "\ndocs:\n" + docs
            }, 
            {
                "role": "user",
                "content": 'question: ' + question + '\ndocs:' + docs
            }
        ]
    )

    docs = response.choices[0].message.content

    return docs, ""

In [7]:
def generate_answer(question, docs):
    query = "question: " + question + "\n\n"
    query += "content: " + docs
    
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "content를 참고하여 question의 답변을 작성해줘"
            }, 
            {
                "role": "user",
                "content": query
            }
        ]
    )

    return response.choices[0].message.content, ""

In [8]:
def hallucination_checker(answer, docs):
    parser = JsonOutputParser()
    
    query = "answer: " + answer + "\n\n"
    query += "docs: " + docs
    
    prompt = PromptTemplate(
        template="""
유저의 answer과 docs가 들어오면, docs에 근거하여 답변이 잘 작성되었는지 확인해줘.
docs에 없는 내용을 지어냈는지 확인해야해.
말을 지어냈다면,  yes
docs에 근거했다면, no
답변해줘

Answer the user query.\n{format_instructions}\n{query}\n
key 값은 hallucination으로 해줘
    """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    chain = prompt | llm | parser
    
    result = chain.invoke({"query": query})

    return result

In [9]:
def final(answer, docs):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "content에서 source를 뽑아서, [answer 내용]\n source: [source], [제목] 형태로 적어줘"
            }, 
            {
                "role": "user",
                "content": 'answer: ' + answer + '\ndocs:' + docs
            }
        ]
    )

    return response.choices[0].message.content

In [10]:
def planning_llm(question):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """
사용자의 질문이 들어오면 관련 정보를 검색하고 답변하는 시스템을 만드려고 해.
너의 역할은 시스템의 현재 단계 정보가 들어오면 다음 단계의 함수로 전달하는 일이야.
입력값이 들어오면 함수의 이름만 반환해줘

[제한사항]
출력 포멧은 json, key 이름은 "function"으로 정확하게 지켜야 합니다.
```json ``` 문자는 제외해야 합니다.
    
[시스템 flow]
    0. start: 처음 시작 단계. 유저가 질문을 입력한 직후.
    
    1. search_retriever: vectorstore에서 사용자 질문과 유사한 정보 검색
    
    2. relevance_checker: 1번 검색 결과가 사용자 질문과 진짜 유사한지 검증. 만약 유사하지 않다면 2-1번으로 돌아감. 유사하다면 3번으로 넘어감.
    출력 예시: {related: yes}

    2-1. select_relevance: 1번 검색 결과에서 사용자 질문과 유사한 정보만 뽑음. 이 작업 이후 2번으로 넘어감.
    
    3. generate_answer: 1번 검색 결과와 사용자 질문을 같이 전달하여, 답변을 만듦.
    
    4. hallucination_checker: 3번 결과와 docs를 비교하여, hallucination이 발생했는지 확인. 만약 할루시에이션이 발생했다면 다시 3번으로 돌아감. 발생하지 않았다면 5번으로 넘어감.
    출력 예시: {hallucination: no}
    
    5. final: 최종적으로 3번의 결과에 출처명 기입
    
[입력값 형태]
{state: start, result: ''}
    
[출력]
    Answer the user query.\n{format_instructions}\n{query}\n
    key 값은 function으로 해줘
        """
            }, 
            {
                "role": "user",
                "content": question
            }
        ]
    )

    return response.choices[0].message.content

In [14]:
def main(question):
    retriever = create_retriever()
    state = "start"
    result = ""
    re_flag = 0
    ha_flag = 0
    docs = ""
    answer = ""

    while(True):
        plan = planning_llm(str({'state': state, 'result': result}))
        state = eval(plan)['function']

        if state == "search_retriever":
            print("#### search_retriever ####")
            docs, result = search_retriever(question, retriever)
            print(result)
            
        elif state == "relevance_checker":
            if re_flag > 1:
                print("#### failed: not relevant ####")
                break;

            print("#### relevance_checker ####")
            result = relevance_checker(question, docs)
            re_flag += 1
            print(result)
            print()

        elif state == "select_relevance":
            print("#### select_relevance ####")
            docs, result = select_relevance(question, docs)
            print(result)
            
        elif state == "generate_answer":
            print("#### generate_answer ####")
            answer, result = generate_answer(question, docs)
            print(result)
            
        elif state == "hallucination_checker":
            print(ha_flag)
            if ha_flag > 1:
                print("#### failed: hallucination ####")
                break;

            print("#### hallucination_checker ####")
            result = hallucination_checker(answer, docs)
            ha_flag += 1
            print(result)
            print()
            
        elif state == "final":
            print("#### final ####")
            result = final(answer, docs)
            print("#### End! ####")
            print(result)
            break;
        else:
            print("#### Error! ####")
            print(plan)
            break

In [15]:
main("agent memory")

#### search_retriever ####

#### relevance_checker ####
[o, o, x, x, x, x]
{'related': 'no'}

#### select_relevance ####
[o, o, x, x, x, o]

#### relevance_checker ####
[o, o, x]
{'related': 'no'}

#### select_relevance ####
[o, o, x]

#### failed: not relevant ####


In [16]:
main("What is CoT")

#### search_retriever ####

#### relevance_checker ####
[o, o, o, x, x]
{'related': 'no'}

#### select_relevance ####
[o, o, o, x, x, x]

#### relevance_checker ####
[o, o, o]
{'related': 'yes'}

#### generate_answer ####

0
#### hallucination_checker ####
{'hallucination': 'no'}

#### final ####
#### End! ####
answer: Chain-of-Thought (CoT) prompting is a technique that involves generating a series of short sentences to explain reasoning process step by step, which is referred to as reasoning chains or rationales. This approach is especially beneficial for complex reasoning tasks, particularly when using large models (those with more than 50 billion parameters). While it offers significant advantages in complicated scenarios, simple tasks see only slight improvements from employing CoT prompting. There are primarily two types of CoT prompts, one of which is few-shot CoT, where the model is prompted with a few examples containing high-quality reasoning chains, either manually written o

In [17]:
main("I like apple")

#### search_retriever ####

#### relevance_checker ####
[o, x, x, x, x]
{'related': 'no'}

#### select_relevance ####
[x, x, x, x, x, x]

#### relevance_checker ####
[]
{'related': 'no'}

#### select_relevance ####
[]

#### failed: not relevant ####
